# Oil Environment Code Demo

This problem, adaptved from [here](https://www.pnas.org/content/109/3/764) is a continuous variant of the “Grid World” environment. It comprises of an agent surveying a d-dimensional map in search of hidden “oil deposits”. The world is endowed with an unknown survey function which encodes the probability of observing oil at that specific location. For agents to move to a new location they pay a cost proportional to the distance moved, and surveying the land produces noisy estimates of the true value of that location. In addition, due to varying terrain the true location the agent moves to is perturbed as a function of the state and action.


There is a $d$-dimensional reinforcement learning environment in the space $X = [0, 1]^d$.  The action space $A = [0,1]^d$ corresponding to the ability to attempt to move to any desired location within the state space.  On top of that, there is a corresponding reward function $f_h(x,a)$ for the reward for moving the agent to that location.  Moving also causes an additional cost $\alpha d(x,a)$ scaling with respect to the distance moved.

In this notebook we run a sample experiment for the setting when $d = 1$ and the reward function is taken to be a quadratic.  We compare several heuristics to existing reinforcement learning algorithms.

In [1]:
import or_suite
import numpy as np

import copy

import os
from stable_baselines3.common.monitor import Monitor
from stable_baselines3 import PPO
from stable_baselines3.ppo import MlpPolicy
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.evaluation import evaluate_policy
import pandas as pd


import gym

Here we use the oil environment as outlined in `or_suite/envs/oil_discovery/oil_environment.py`.  The package has default specifications for all of the environments in the file `or_suite/envs/env_configs.py`, and so we use one the defaults.

In addition, we need to specify the number of episodes for learning, and the number of iterations (in order to plot average results with confidence intervals).

In [2]:
CONFIG =  or_suite.envs.env_configs.oil_environment_default_config

epLen = CONFIG['epLen']
nEps = 100
numIters = 2

epsilon = (nEps * epLen)**(-1 / 4)
action_net = np.arange(start=0, stop=1, step=epsilon)
state_net = np.arange(start=0, stop=1, step=epsilon)

scaling_list = [0.1, 0.3, 1, 5]

DEFAULT_SETTINGS = {'seed': 1, 
                    'recFreq': 1, 
                    'dirPath': '../data/oil/', 
                    'deBug': False, 
                    'nEps': nEps, 
                    'numIters': numIters, 
                    'saveTrajectory': True, 
                    'epLen' : 5,
                    'render': False,
                    'pickle': False
                    }



In [3]:
oil_env = gym.make('Oil-v0', config=CONFIG)
mon_env = Monitor(oil_env)
dim = CONFIG['dim']
cost_param = CONFIG['cost_param']
prob = CONFIG['oil_prob']

agents = { 'SB PPO': PPO(MlpPolicy, mon_env, gamma=1, verbose=0, n_steps=epLen),
'Random': or_suite.agents.rl.random.randomAgent(),
'AdaQL': or_suite.agents.rl.ada_ql.AdaptiveDiscretizationQL(epLen, scaling_list[0], True, dim*2),
'AdaMB': or_suite.agents.rl.ada_mb.AdaptiveDiscretizationMB(epLen, scaling_list[0], 0, 2, True, True, dim, dim),
'Unif QL': or_suite.agents.rl.enet_ql.eNetQL(action_net, state_net, epLen, scaling_list[0], (dim,dim)),
'Unif MB': or_suite.agents.rl.enet_mb.eNetMB(action_net, state_net, epLen, scaling_list[0], (dim,dim), 0, False),
}

path_list_line = []
algo_list_line = []
path_list_radar = []
algo_list_radar= []
for agent in agents:
    print(agent)
    DEFAULT_SETTINGS['dirPath'] = '../data/oil_metric_'+str(agent)+'_'+str(dim)+'_'+str(cost_param)+'_'+str(prob.__name__)+'/'
    if agent == 'SB PPO':
        or_suite.utils.run_single_sb_algo(mon_env, agents[agent], DEFAULT_SETTINGS)
    elif agent == 'AdaQL' or agent == 'Unif QL' or agent == 'AdaMB' or agent == 'Unif MB':
        or_suite.utils.run_single_algo_tune(oil_env, agents[agent], scaling_list, DEFAULT_SETTINGS)
    else:
        or_suite.utils.run_single_algo(oil_env, agents[agent], DEFAULT_SETTINGS)

    path_list_line.append('../data/oil_metric_'+str(agent)+'_'+str(dim)+'_'+str(cost_param)+'_'+str(prob.__name__))
    algo_list_line.append(str(agent))
    if agent != 'SB PPO':
        path_list_radar.append('../data/oil_metric_'+str(agent)+'_'+str(dim)+'_'+str(cost_param)+'_'+str(prob.__name__))
        algo_list_radar.append(str(agent))

fig_path = '../figures/'
fig_name = 'oil_metric'+'_'+str(dim)+'_'+str(cost_param)+'_'+str(prob.__name__)+'_line_plot'+'.pdf'
or_suite.plots.plot_line_plots(path_list_line, algo_list_line, fig_path, fig_name, int(nEps / 40)+1)

additional_metric = {}
fig_name = 'oil_metric'+'_'+str(dim)+'_'+str(cost_param)+'_'+str(prob.__name__)+'_radar_plot'+'.pdf'
or_suite.plots.plot_radar_plots(path_list_radar, algo_list_radar,
fig_path, fig_name,
additional_metric
)

/home/sean/anaconda3/envs/ORSuite/lib/python3.7/site-packages/stable_baselines3/ppo/ppo.py:138: UserWarning: You have specified a mini-batch size of 64, but because the `RolloutBuffer` is of size `n_steps * n_envs = 5`, after every 0 untruncated mini-batches, there will be a truncated mini-batch of size 5
We recommend using a `batch_size` that is a factor of `n_steps * n_envs`.
Info: (n_steps=5 and n_envs=1)
  f"You have specified a mini-batch size of {batch_size},"


SB PPO
Writing to file ../data/oil_metric_SB PPO_1_0_<lambda>/data.csv
Random
Writing to file data.csv
AdaQL
Writing to file data.csv
0.1
AdaMB
Writing to file data.csv
0.3
Unif QL
Writing to file data.csv
0.3
Unif MB
Writing to file data.csv
0.1
  Algorithm    Reward      Time   Space
0    Random  3.676325  7.184680 -4934.0
1     AdaQL  4.867660  6.875450 -5580.0
2     AdaMB  4.816125  6.924240 -4620.0
3   Unif QL  4.167185  6.770350 -4584.0
4   Unif MB  5.000000  6.204685 -4628.0


Here we see the uniform discretization model based algorithm performs the best with a minimal time complexity for evaluating the algorithm.